In [ ]:
from pathlib import Path

import rioxarray as rxr
import xarray as xr
from rasterio.enums import Resampling

substrate_dir_path = Path("~/onnx_models/substrate_20m_datapackage").expanduser()

substrate_files = [
    next(substrate_dir_path.glob("NCC_substrate_20m.tif")),
    next(substrate_dir_path.glob("SOG_substrate_20m.tif")),
    next(substrate_dir_path.glob("WCVI_substrate_20m.tif")),
    next(substrate_dir_path.glob("QCS_substrate_20m.tif")),
    next(substrate_dir_path.glob("HG_substrate_20m.tif")),
]

# Reproject each to match stacked (only processes the Sentinel extent)
substrate_gen = map(rxr.open_rasterio, substrate_files)

In [ ]:
d0 = next(substrate_gen)
substrate_data = [d0]
for i, substrate in enumerate(substrate_gen):
    dx = substrate.rio.reproject_match(d0, resampling=Resampling.bilinear)
    substrate_data.append(dx)

substrate = xr.concat(substrate_data, dim="band")

substrate

In [ ]:
# Merge: later files overwrite earlier ones where both have valid values
valid_values = [1, 2, 3, 4]
merged_substrate = xr.zeros_like(substrate_data[0])

for substrate in substrate_data:
    valid_mask = substrate.isin(valid_values)
    merged_substrate = xr.where(valid_mask, substrate, merged_substrate)

# Fill remaining with 0
merged_substrate = merged_substrate.fillna(0)

merged_substrate = merged_substrate.rio.write_crs(d0.rio.crs)

In [ ]:
merged_substrate.rio.to_raster(
    Path("~/onnx_models/substrate_20m.tif").expanduser(),
    driver="GTiff",
    compress="lzw",
    tiled=True,
    blockxsize=256,
    blockysize=256,
    interleave="pixel",
    photometric="MINISBLACK",
)